In [92]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

from scipy.optimize import minimize, rosen, rosen_der

## Group Assignment
### Team Number: 06
### Team Member Names: Shayan Jalali, Krish Suryavanshi, Paul Reddy
### Team Strategy Chosen: Market Meet

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [93]:
# Krish's code

csv_file = "Tickers_Example.csv"
tickers = pd.read_csv(csv_file, header=None, names=["Ticker"])

# Include only valid US and Canadian companies that have listed stocks in these markets.

start_date = "2024-10-01"
end_date = "2025-10-01" # TO INCLUDE SEPT 30TH

valid_tickers = []

for ticker in tickers["Ticker"]:
    hist = yf.Ticker(ticker).history(start=start_date, end=end_date)
    
    if not hist.empty:
        valid_tickers.append(ticker)

# Exclude any stock with average daily volume < 5,000 shares between Oct 1, 2024 and Sep 30, 2025 (drop months with < 18 trading days).

volume_cutoff = 5000
minimum_trading_days = 18
final_tickers = []

for valid_ticker in valid_tickers:

    hist = yf.Ticker(valid_ticker).history(start=start_date, end=end_date)
    months = list(hist.index.strftime("%Y-%m"))

    unique_months = []
    for month in months:
        if month not in unique_months:
            unique_months.append(month)
    
    good_months = []
    for unique_month in unique_months:
        if months.count(unique_month) >= minimum_trading_days:
            good_months.append(unique_month)
    
    mask = hist.index.strftime("%Y-%m").isin(good_months)
    filtered_hist = hist[mask]

    avg_volume = filtered_hist["Volume"].mean()
    if avg_volume >= volume_cutoff:
        final_tickers.append(valid_ticker)

# create closing prices DataFrame for the filtered tickers

close_prices = pd.DataFrame()
for final_ticker in final_tickers:
    hist = yf.Ticker(final_ticker).history(start=start_date, end=end_date)
    close_prices[final_ticker] = hist["Close"]

close_prices.index = close_prices.index.strftime("%Y-%m-%d")

close_prices

$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found


,AAPL,ABBV,ABT,ACN,AGN.CN,AIG,AMZN,AXP,BA,BAC,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-01,224.944046,188.798172,110.925461,345.618683,0.0800,71.177109,185.130005,265.044800,154.220001,38.318249,...,162.130707,160.253281,105.980003,21.057085,81.035973,193.983368,570.918213,239.846207,123.182945,42.547165
2024-10-02,225.510849,188.663986,111.003609,349.850769,0.0800,70.912834,184.759995,267.334106,152.889999,38.328014,...,164.614807,159.121506,106.830002,20.899042,80.413277,195.744217,579.737000,237.697327,122.942627,42.346203
2024-10-03,224.407043,187.350723,109.626312,354.711304,0.0800,70.805168,181.960007,265.034943,150.520004,38.347557,...,165.201599,157.817078,105.980003,20.713108,80.941628,194.050720,580.167603,235.040527,120.825966,42.087818
2024-10-04,225.530716,186.238800,110.026794,355.693207,0.0825,74.436455,186.509995,273.028809,155.000000,39.187786,...,165.182007,159.409256,111.910004,20.555063,81.620941,195.051422,578.650635,233.311615,121.315842,42.776840
2024-10-07,220.449341,185.586990,110.730095,350.351532,0.0800,72.077583,180.800003,270.872040,155.910004,39.041233,...,163.265167,159.121506,110.190002,20.601547,82.017204,193.665817,571.172791,231.367828,121.269630,42.556740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-24,252.065643,218.415421,133.297256,239.080002,0.0700,77.900002,220.210007,339.619476,215.100006,51.700001,...,173.550003,203.245789,206.839996,21.969999,107.572151,182.808289,351.809998,230.360001,82.466415,48.657986
2025-09-25,256.621216,216.975800,132.719818,232.559998,0.0700,77.099998,218.149994,339.320221,213.529999,51.849998,...,169.679993,202.332642,199.960007,21.820000,108.929314,180.429520,345.559998,232.000000,81.168968,48.509575
2025-09-26,255.212601,219.030991,132.958771,238.970001,0.0700,77.980003,219.779999,340.816437,221.259995,52.209999,...,169.199997,202.114273,195.399994,21.830000,109.315651,182.917328,344.079987,235.199997,82.289490,48.915222


In [94]:
# Paul's code

#Get the history of the S&P500 and TSX
sp_data = yf.Ticker("^GSPC").history(start=start_date, end=end_date)
tsx_data = yf.Ticker("^GSPTSE").history(start=start_date, end=end_date)
sp_close = sp_data["Close"]
tsx_close = tsx_data["Close"]

#Calculate cumulative total returns 
sp_returns = sp_close.pct_change()
tsx_returns = tsx_close.pct_change()
sp_total_return = sp_returns.cumsum()*100
tsx_total_return = tsx_returns.cumsum()*100

#Average
benchmark_avg = (sp_total_return + tsx_total_return) / 2

#Store them in dataframes
sp_df = pd.DataFrame({"S&P500 TotalReturn": sp_total_return})
tsx_df = pd.DataFrame({"TSX TotalReturn": tsx_total_return})
benchmark_df = pd.DataFrame({"Benchmark Average": benchmark_avg})

benchmark_df.index = benchmark_df.index.strftime("%Y-%m-%d")

print("Benchmark Average:")

#Calculate cumulative total returns for stocks
stock_returns = close_prices.pct_change()
stock_total_returns = stock_returns.cumsum() * 100


#Merge the returns of the stocks and the benchmark average
merged = pd.merge(stock_total_returns, benchmark_df, on="Date", how="inner")

#Store the average distances between the returns of the stock to the returns of the benchmark average in a distances dataframe
distances_dict = {}
for ticker in close_prices.columns:
    avg_distance = (merged[ticker] - merged["Benchmark Average"]).abs().mean()
    distances_dict[ticker] = avg_distance

distances_df = pd.DataFrame(list(distances_dict.items()), columns=["Ticker", "Average Distance"])
distances_df = distances_df.sort_values(by="Average Distance", ascending=True).reset_index(drop=True)
distances_df



Benchmark Average:


C:\Users\s2jal\AppData\Local\Temp\ipykernel_18772\3423409461.py:28: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = close_prices.pct_change()


,Ticker,Average Distance
0,RY.TO,1.616308
1,BLK,4.284486
2,USB,4.455741
3,AXP,6.071996
4,AIG,6.400783
5,AAPL,6.909028
6,ABT,7.203750
7,ABBV,7.267377
8,QCOM,7.435325
9,T.TO,7.546112


In [95]:
# Shayan's code portfolio construction 

# get live USD/CAD exchange rate
exchange_rate_ticker = yf.Ticker("CAD=X")
exchange_rate_data = exchange_rate_ticker.history(period="1d")
exchange_rate = exchange_rate_data['Close'].iloc[-1]
print(f"Current USD/CAD exchange rate: {exchange_rate}")

# categorize all available stocks by market cap
temp_large = []
temp_small = []

for ticker in distances_df['Ticker']:
    try:
        stock = yf.Ticker(ticker)
        mc = stock.info.get('marketCap', 0)
        curr = stock.info.get('currency', 'USD')
         
        mc_cad = mc * exchange_rate if curr == 'USD' else mc
        
        if mc_cad >= 10e9:
            temp_large.append(ticker)
        elif mc_cad < 2e9:
            temp_small.append(ticker)
    except:
        continue

# ensure we have at least 1 large and 1 small
if len(temp_large) == 0 or len(temp_small) == 0:
    raise Exception(f"Cannot meet requirements: found {len(temp_large)} large-cap, {len(temp_small)} small-cap (need at least 1 of each)")

# force 1 large and 1 small into selection
selected_tickers = []
selected_tickers.append(temp_large[0])
selected_tickers.append(temp_small[0])

# get sectors for the 2 already selected
sector_counts_selected = {}
for ticker in selected_tickers:
    try:
        stock = yf.Ticker(ticker)
        sector = stock.info.get('sector', 'Unknown')
        sector_counts_selected[sector] = sector_counts_selected.get(sector, 0) + 1
    except:
        pass

# fill remaining slots with best trackers respecting 40% sector limit
target_n = 20
max_sector_frac = 0.4

for ticker in distances_df["Ticker"]:
    if ticker in selected_tickers:
        continue
    if len(selected_tickers) >= target_n:
        break
    
    if ticker not in temp_large and ticker not in temp_small:
        continue
    
    try:
        stock = yf.Ticker(ticker)
        sector = stock.info.get('sector', 'Unknown')
        
        new_total = len(selected_tickers) + 1
        new_sector_count = sector_counts_selected.get(sector, 0) + 1
        new_sector_frac = new_sector_count / new_total
        
        if new_sector_frac > max_sector_frac:
            continue
        
        selected_tickers.append(ticker)
        sector_counts_selected[sector] = new_sector_count
    except:
        continue

print(f"\nSelected {len(selected_tickers)} stocks")
print(f"Large-caps: {len([t for t in selected_tickers if t in temp_large])}")
print(f"Small-caps: {len([t for t in selected_tickers if t in temp_small])}")
print(f"Sector counts: {sector_counts_selected}")

# create stock_info dataframe
ticker_list = []
sector_list = []
market_cap_cad_list = []
cap_type_list = []

for ticker in selected_tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        sector = info.get('sector', 'Unknown')
        market_cap = info.get('marketCap', 0)
        currency = info.get('currency', 'USD')
        
        if currency == 'USD':
            market_cap_cad = market_cap * exchange_rate
        else:
            market_cap_cad = market_cap
        
        if market_cap_cad >= 10e9:
            cap_type = 'Large'
        elif market_cap_cad < 2e9:
            cap_type = 'Small'
        else:
            cap_type = 'Mid'
        
        ticker_list.append(ticker)
        sector_list.append(sector)
        market_cap_cad_list.append(market_cap_cad)
        cap_type_list.append(cap_type)
    except:
        continue

stock_info = pd.DataFrame({
    'Ticker': ticker_list,
    'Sector': sector_list,
    'MarketCap_CAD': market_cap_cad_list,
    'CapType': cap_type_list
})

print("\n", stock_info)

Current USD/CAD exchange rate: 1.408400058746338

Selected 20 stocks
Large-caps: 19
Small-caps: 1
Sector counts: {'Financial Services': 5, 'Healthcare': 4, 'Technology': 3, 'Communication Services': 1, 'Industrials': 3, 'Consumer Cyclical': 1, 'Consumer Defensive': 3}

     Ticker                  Sector  MarketCap_CAD CapType
0    RY.TO      Financial Services   2.933155e+11   Large
1   AGN.CN              Healthcare   2.911778e+06   Small
2     AAPL              Technology   5.564946e+12   Large
3     T.TO  Communication Services   2.916432e+10   Large
4    TD.TO      Financial Services   1.974993e+11   Large
5      TXN              Technology   1.963282e+11   Large
6      CAT             Industrials   3.617519e+11   Large
7     AMZN       Consumer Cyclical   3.269282e+12   Large
8      BAC      Financial Services   5.320292e+11   Large
9       MO      Consumer Defensive   1.376515e+11   Large
10      PG      Consumer Defensive   4.884846e+11   Large
11     UNP             Industrial

In [96]:

# get the daily returns for the selected stocks, and dropping the rows that dont
selected_stock_returns = stock_returns[selected_tickers].dropna()

# gets the benchmark daily returns: going to be useful when using scipy optimization
benchmark_daily_returns = (sp_returns + tsx_returns) / 2

# have to use strftime otherwise get mismatch when comparing to the aligned stock returns index
benchmark_daily_returns.index = benchmark_daily_returns.index.strftime('%Y-%m-%d')

# find the cmommon dates to use with .index intersection() for both the stock returns and benchmark returns (had to alisgn them)
common_dates = selected_stock_returns.index.intersection(benchmark_daily_returns.index)
aligned_stock_returns = selected_stock_returns.loc[common_dates]
aligned_benchmark_returns = benchmark_daily_returns.loc[common_dates]


# using the covariance on alisgned selected stock rteurns to generate matrix
cov_matrix = aligned_stock_returns.cov().values # with the data for now, matrix is 20x20 numpy array with covariances between all pairs of selected stocks
# print(cov_matrix)

# calculate covariances between each stock and the benchmark
stock_benchmark_covariances = []

# for stock in the list of stocks we've filtered out
for ticker in selected_tickers:
    # assign series (1D array) to the common dates of the stock
    stock_returns_series = aligned_stock_returns[ticker]
    # assign series (1D array) to the common dates of the benchmark returns
    benchmark_returns_series = aligned_benchmark_returns

    # calucluate the covariance at that specific point (daily values)
    covariance = stock_returns_series.cov(benchmark_returns_series)
    # add it to stock_benchmark relationship
    stock_benchmark_covariances.append(covariance)

# store as numpy array for scipy on all the covariances that we've appdned to the empty list  before for loop (expects num array and also want consistensy with cov_matrix)
benchmark_cov = np.array(stock_benchmark_covariances)
    
# of the benchmark returns that are aligned with the common dates, how much does it go up and oown daily 
benchmark_var = aligned_benchmark_returns.var()

# print to verify all three covariance components
print("Covariance matrix shape:", cov_matrix.shape) # how the stocks move relative to eadch other 
print("Benchmark covariances length:", len(benchmark_cov)) # how the stocks move with the benchmark
print("Benchmark variance:", benchmark_var) # how the becnhmark moves

Covariance matrix shape: (20, 20)
Benchmark covariances length: 20
Benchmark variance: 9.698255889153167e-05


In [97]:
# Shayan's code

# define function that calculates tracking error variance
# input: weights (array of 20 numbers)
# calculate: portfolio_var = weights^T @ cov_matrix @ weights
# calculate: portfolio_bench_cov = weights^T @ benchmark_cov
# calculate: tracking_error_var = portfolio_var - 2*portfolio_bench_cov + benchmark_var
# return: tracking_error_var

def tracking_error_variance(weights):
    # inner dot product understands how the stocks amplify each other, and outer dot product determines weightages based on this interactions
    portfolio_var = np.dot(weights, np.dot(cov_matrix, weights))
    portfolio_bench_cov = np.dot(weights, benchmark_cov)
    # https://quant.stackexchange.com/questions/35720/ex-ante-tracking-error-active-strategies-and-the-size-of-the-covariance-matrix used this interesting source for calculating tracking error
    tracking_error_var = portfolio_var - 2 * portfolio_bench_cov + benchmark_var

    # calculate portfolio beta and add penalty to encourage beta closer to 1.0 (tracking error does its job but the restrictions maade it have lower beta)
    portfolio_beta = portfolio_bench_cov / benchmark_var
    beta_penalty = 500 * (portfolio_beta - 1.0)**2

    return tracking_error_var + beta_penalty

# constraint 1: all weights must sum to 1 (100%)
# constraint 2: each weight must be >= minimum (100 / (2*n) percent)
# constraint 3: each weight must be <= 0.15 (15%)

num_stocks = len(selected_tickers) # get the number of stocks we selected
min_weight_stocks = (100/(2 * num_stocks)) / 100  # get the minimum weight per stock

# make dict to have each ticker with its sector
sector_lookup = {}
for i in range(len(stock_info)):
    t = stock_info.loc[i, "Ticker"]
    sector_lookup[t] = stock_info.loc[i, "Sector"]

# get each sector with the indexes of its stocks in the weights list
sector_index_map = {}
for i in range(len(selected_tickers)):
    t = selected_tickers[i]
    sector = sector_lookup[t]
    if sector not in sector_index_map:
        sector_index_map[sector] = []
    sector_index_map[sector].append(i)

# define constratints function

# this will verify with scipy that a function returns 0 when constraint is satsiifed 
def weights_must_sum_to_one(weights):
    return np.sum(weights) - 1

constraints = [{'type': 'eq', 'fun': weights_must_sum_to_one}]

# do sector check per sector by summing to get the weight of each sector
for sector in sector_index_map:
    idx_list = sector_index_map[sector]

    def sector_constraint(weights, idx_list=idx_list):
        sector_weight = 0
        for j in idx_list:
            sector_weight = sector_weight + weights[j]
        return 0.40 - sector_weight   # must be >= 0

    constraints.append({'type': 'ineq', 'fun': sector_constraint})

# create bounds list
bounds = []
for ticker in selected_tickers:
# boudns requires (min, max) pairs for each element based on scipy documentation. pairs = tuples also want them to be immutasble
    bounds.append((min_weight_stocks, 0.15))

# create initial_weights arrayz; op[timizer needs a starting point before caluclating based on the bounds which have been made
# x0ndarray, shape (n,)
# Initial guess. Array of real elements of size (n,), where n is the number of independent variables.
# Reqwuired to pass in x0. Basically the initial guess based on scipy documentation

# get distances for the selected tickers in the same order
dist_for_port = distances_df[distances_df["Ticker"].isin(selected_tickers)]
dist_for_port = dist_for_port.set_index("Ticker")

distance_list = []
for t in selected_tickers:
    distance_list.append(dist_for_port.loc[t, "Average Distance"])

distance_array = np.array(distance_list)

# closer to benchmark (small distance) = bigger score
scores = 1.0 / distance_array

# base weights proportional to these scores
scores_sum = scores.sum()
raw_weights = scores / scores_sum # normalize it to 1

clipped_weights = np.clip(raw_weights, min_weight_stocks, 0.15) # .clip allows us to set min and max values

initial_weights = clipped_weights / clipped_weights.sum() # normalize it to 1

# contains everything about how result went based on all the data fed into it
# using SLSQP method because it handles bounds and equality constraints efficiently
# extracts the optimal weights (x attribute) from the optimizer result object
optimizer_data= minimize(tracking_error_variance, initial_weights, method='SLSQP', bounds = bounds, constraints=constraints)

# check for optimization success
if not optimizer_data.success:
    print(f"Optimization failed: {optimizer_data.message}")
    raise Exception("Cannot find valid portfolio weights")


# sintially needed the x0 value, now calling the actual .x value from the optimizer 
optimal_weights = optimizer_data.x


In [98]:
# Shayan's code

# checks if optimization worked
print("Success:", optimizer_data.success)
print("Tracking error variance:", optimizer_data.fun)
print("Total weight:", np.sum(optimal_weights))

print("\nWeights by stock:")
# make counter variable 
i = 0
# make for loop that itrates over the sleected ticker list
for ticker in selected_tickers:
    # the ticker weightage is the optimal weights index * 100 to convert to percentage for each ticker and then rounds to 2 decimal places
    print(f"{ticker}: {optimal_weights[i]*100:.2f}%")
    i += 1

Success: True
Tracking error variance: 2.3050638242454517e-05
Total weight: 1.0

Weights by stock:
RY.TO: 9.15%
AGN.CN: 2.85%
AAPL: 10.64%
T.TO: 4.43%
TD.TO: 4.31%
TXN: 10.01%
CAT: 3.98%
AMZN: 9.75%
BAC: 3.73%
MO: 3.65%
PG: 3.65%
UNP: 3.57%
BMY: 3.49%
KO: 3.45%
PYPL: 9.30%
LLY: 2.93%
PFE: 2.91%
ACN: 2.76%
BK: 2.76%
BA: 2.72%


In [104]:
# calculate portfolio beta relative to benchmark
# beta measures how much portfolio moves relative to benchmark

# get portfolio returns using optimal weights (use ALIGNED returns)
portfolio_returns = (aligned_stock_returns * optimal_weights).sum(axis=1)

# calculate covariance  between portfolio and benchmark (use ALIGNED benchmark)
portfolio_benchmark_cov = portfolio_returns.cov(aligned_benchmark_returns)

# calculate variance of benchmark (use ALIGNED benchmark)
benchmark_variance = aligned_benchmark_returns.var()

# beta = Cov(portfolio, benchmark) / Var(benchmark)
portfolio_beta = portfolio_benchmark_cov / benchmark_variance

print(f"\nPortfolio Beta: {portfolio_beta}")


Portfolio Beta: 0.9999999622702493


In [100]:
# for step 5: initial check of like constrains of 40% was to identify if the stocks selected were diverse enoughh
# after optimization, we're going to get weightages htat violate those retsirtcions so redoing the 40% restrictions here will help with ensuring that they fit the constrants
# even if those contsraints make our weightages worse, we have to follow the restrictions 

# krish's code

# make a df for tickers and weights
dict = {"Ticker": selected_tickers, "Weight": optimal_weights}
weights_df = pd.DataFrame(dict)

# create portfolio df
portfolio_df = pd.merge(weights_df, stock_info , on="Ticker", how="inner")

# get sector weights
sector_weights_series = portfolio_df.groupby("Sector")["Weight"].sum()
sector_weights = sector_weights_series.to_dict()

# print them
for sector in sector_weights:
    print(str(sector) + ": " + str(np.round(sector_weights[sector] * 100, 2)) + "%")

print()

# check for sector violations (max 40% in one sector)
sector_violations = False
tolerance = 0.0001  # small tolerance to account for python's precision issues w/ numbers
for sector in sector_weights:
    if sector_weights[sector] > 0.4 + tolerance:
        sector_violations = True
        print("Sector weight violation: " + str(sector) + " is " + str(np.round(sector_weights[sector] * 100, 2)) + "%; max 40%")

if not sector_violations:
    print("No sector weight violations.")

print()

# check for cap violations (need one large cap and one small cap minimum)
large_rows = portfolio_df[portfolio_df["CapType"] == "Large"]
large_weight_total = large_rows["Weight"].sum()

small_rows = portfolio_df[portfolio_df["CapType"] == "Small"]
small_weight_total = small_rows["Weight"].sum()

print("Total Large Cap Weight: " + str(np.round(large_weight_total * 100, 2)) + "%")
print("Total Small Cap Weight: " + str(np.round(small_weight_total * 100, 2)) + "%")
print()

if (not large_weight_total > 0) or (not small_weight_total > 0):
    print("Market cap weight violation; need at least one large cap and one small cap stock")

Communication Services: 4.43%
Consumer Cyclical: 9.75%
Consumer Defensive: 10.74%
Financial Services: 29.24%
Healthcare: 12.17%
Industrials: 10.26%
Technology: 23.4%

No sector weight violations.

Total Large Cap Weight: 97.15%
Total Small Cap Weight: 2.85%



In [101]:
# Paul's code

#create a dictionary with the ticker and its price and currency so we can convert for fees
price_data = {
    "Ticker": [],
    "Price": [],
    "Currency": []
}

for index, row in portfolio_df.iterrows():
    ticker = row["Ticker"]   

    try:
        stock = yf.Ticker(ticker)

        # get most recent avilable closing price (in this case when the code will fetch Nov 21st)
        hist = stock.history(period="5d")

        if hist.empty:
            print(f"No recent price data for {ticker}")
            continue

        close_price = hist["Close"].iloc[-1]

        # extract currency
        currency = stock.info.get("currency", "Unknown")

        # add row to the df structure
        price_data["Ticker"].append(ticker)
        price_data["Price"].append(close_price)
        price_data["Currency"].append(currency)


    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# convert to DataFrame
prices_df = pd.DataFrame(price_data)

print("\nFinal Prices DataFrame:")
prices_df



Final Prices DataFrame:


,Ticker,Price,Currency
0,RY.TO,207.979996,CAD
1,AGN.CN,0.085000,CAD
2,AAPL,266.250000,USD
3,T.TO,18.840000,CAD
4,TD.TO,114.660004,CAD
5,TXN,153.330002,USD
6,CAT,546.130005,USD
7,AMZN,217.139999,USD
8,BAC,51.000000,USD
9,MO,58.180000,USD


In [102]:
# calculate shares with transaction fees

# merge prices into portfolio
portfolio_with_prices = pd.merge(portfolio_df, prices_df, on="Ticker", how="inner")

# start with a budget of 1 million CAD
budget_cad = 1000000

# calculate initial target values based on optimal weights
portfolio_with_prices["TargetValue_CAD"] = portfolio_with_prices["Weight"] * budget_cad

# convert prices to CAD if necessary
target_values_stock_currency = []
for index, row in portfolio_with_prices.iterrows():
    if row["Currency"] == "USD":
        target_values_stock_currency.append(row["TargetValue_CAD"] / exchange_rate)
    else:
        target_values_stock_currency.append(row["TargetValue_CAD"])

portfolio_with_prices["TargetValue_StockCurrency"] = target_values_stock_currency

# calculate initial shares before fees
portfolio_with_prices["Shares"] = portfolio_with_prices["TargetValue_StockCurrency"] / portfolio_with_prices["Price"]

# calculate TOTAL fee for ENTIRE portfolio (not per stock)
total_shares = portfolio_with_prices["Shares"].sum()
total_fee_usd = min(2.15, 0.001 * total_shares)
total_fee_cad = total_fee_usd * exchange_rate

print(f"\nTotal shares across portfolio: {total_shares:.2f}")
print(f"Transaction fee (USD): ${total_fee_usd:.2f}")
print(f"Transaction fee (CAD): ${total_fee_cad:.2f}")

# calculate investable amount after fees
investable_amount_cad = budget_cad - total_fee_cad
print(f"Investable amount after fees (CAD): ${investable_amount_cad:.2f}")

# scale all positions proportionally to fit investable amount
scaling_factor = investable_amount_cad / budget_cad
print(f"Scaling factor: {scaling_factor:.6f}")

# apply scaling to get final shares
portfolio_with_prices["FinalShares"] = portfolio_with_prices["Shares"] * scaling_factor

# calculate final values in stock currency
portfolio_with_prices["FinalValue_StockCurrency"] = portfolio_with_prices["FinalShares"] * portfolio_with_prices["Price"]

# convert final values to CAD
final_values_cad = []
for index, row in portfolio_with_prices.iterrows():
    if row["Currency"] == "USD":
        final_values_cad.append(row["FinalValue_StockCurrency"] * exchange_rate)
    else:
        final_values_cad.append(row["FinalValue_StockCurrency"])

portfolio_with_prices["FinalValue_CAD"] = final_values_cad

# calculate final weights (basically weigfht of each stock in the final portfolio)
portfolio_with_prices["FinalWeight"] = portfolio_with_prices["FinalValue_CAD"] / investable_amount_cad

# verify totals
total_portfolio_value = portfolio_with_prices["FinalValue_CAD"].sum()
total_weights = portfolio_with_prices["FinalWeight"].sum()

print(f"\nTotal Portfolio Value (CAD): ${total_portfolio_value:.2f}")
print(f"Total of Final Weights: {total_weights:.6f}")
print(f"Difference from investable amount: ${abs(total_portfolio_value - investable_amount_cad):.2f}")

# display final portfolio
print("\nFinal Portfolio:")
print(portfolio_with_prices[["Ticker", "Price", "Currency", "FinalShares", "FinalValue_CAD", "FinalWeight"]])


Total shares across portfolio: 343890.43
Transaction fee (USD): $2.15
Transaction fee (CAD): $3.03
Investable amount after fees (CAD): $999996.97
Scaling factor: 0.999997

Total Portfolio Value (CAD): $999996.97
Total of Final Weights: 1.000000
Difference from investable amount: $0.00

Final Portfolio:
    Ticker        Price Currency    FinalShares  FinalValue_CAD  FinalWeight
0    RY.TO   207.979996      CAD     439.750131    91459.230368     0.091460
1   AGN.CN     0.085000      CAD  335090.807466    28482.718934     0.028483
2     AAPL   266.250000      USD     283.672015   106373.180638     0.106374
3     T.TO    18.840000      CAD    2353.827244    44346.105627     0.044346
4    TD.TO   114.660004      CAD     376.233747    43138.962784     0.043139
5      TXN   153.330002      USD     463.301695   100049.993500     0.100050
6      CAT   546.130005      USD      51.708756    39772.799722     0.039773
7     AMZN   217.139999      USD     318.695893    97463.573990     0.097464
8 

In [103]:
# create portfolio with the required columns
final_portfolio = portfolio_with_prices[["Ticker", "Price", "Currency", "FinalShares", "FinalValue_CAD", "FinalWeight"]].copy()

# rename the columns to fit requirements
final_portfolio = final_portfolio.rename(columns={
    "FinalShares": "Shares",
    "FinalValue_CAD": "Value",
    "FinalWeight": "Weight"
})

# convert the weightages to a percentage
final_portfolio["Weight"] = final_portfolio["Weight"] * 100

# reset index to start at 1
final_portfolio.index = range(1, len(final_portfolio) + 1)

# display the final portfolio
print("\nFinal Portfolio DataFrame:")
print(final_portfolio)

# print relevant data about value and weights
total_value = final_portfolio["Value"].sum()
total_weight = final_portfolio["Weight"].sum()

print(f"\nTotal Portfolio Value (CAD): ${total_value:.2f}")
print(f"Total Portfolio Weight (%): {total_weight:.2f}%")
print(f"Budget adjusted for fees (CAD): ${investable_amount_cad:.2f}")

# make a dataframe with the ticker and shares purchased 
shares_purchased_df = final_portfolio[["Ticker", "Shares"]].copy()

# export to csv
shares_purchased_df.to_csv("Stocks_Group_06.csv", index=False)

print("\nShares purchased exported to Stocks_Group_06.csv")
print(f"Portfolio contains {len(final_portfolio)} stocks.")


Final Portfolio DataFrame:
    Ticker        Price Currency         Shares          Value     Weight
1    RY.TO   207.979996      CAD     439.750131   91459.230368   9.145951
2   AGN.CN     0.085000      CAD  335090.807466   28482.718934   2.848281
3     AAPL   266.250000      USD     283.672015  106373.180638  10.637350
4     T.TO    18.840000      CAD    2353.827244   44346.105627   4.434624
5    TD.TO   114.660004      CAD     376.233747   43138.962784   4.313909
6      TXN   153.330002      USD     463.301695  100049.993500  10.005030
7      CAT   546.130005      USD      51.708756   39772.799722   3.977292
8     AMZN   217.139999      USD     318.695893   97463.573990   9.746387
9      BAC    51.000000      USD     519.205095   37293.672793   3.729379
10      MO    58.180000      USD     444.995294   36463.232901   3.646334
11      PG   148.190002      USD     174.655204   36452.428774   3.645254
12     UNP   221.210007      USD     114.468139   35662.798986   3.566291
13     BMY